# Template: visualize imaging data with Vitessce

## Code to change

Places where you will need to edit the code are marked by `# TODO(template)` comments.

In [176]:
import os
import json
from os.path import join
from tifffile import imread
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    OmeTiffWrapper,
    BASE_URL_PLACEHOLDER,
)
from vitessce.data_utils import (
    rgb_img_to_ome_tiff,
    multiplex_img_to_ome_tiff,
)

## Variables to fill in

In [177]:
# TODO(template): specify the file path for the input TIFF image
PATH_TO_INPUT_TIFF = join('..', 'tutorials', 'imaging', 'raw_data', 'dog.tiff')

# TODO(template): specify some file paths for the converted OME-TIFF files
PATH_TO_OUTPUT_OME_TIFF = join('.', 'processed_data', 'imaging', 'output.ome.tif')
PATH_TO_OUTPUT_PYRAMIDAL_OME_TIFF = join('.', 'processed_data', 'imaging' 'output.pyramid.ome.tif')

# TODO(template): this folder should not yet exist, but will be created in step 3.
PATH_TO_EXPORT_DIRECTORY = join('.', 'exported_imaging_data')

# TODO(template): provide names and descriptions
CONFIG_NAME = 'My config'
CONFIG_DESCRIPTION = 'This dataset reveals...'
DATASET_NAME = 'My dataset'
IMG_NAME = 'My image'

## 1. Convert TIFF to OME-TIFF

In [159]:
img_arr = imread(PATH_TO_TIFF)
img_arr.shape

(500, 750, 4)

### Transpose and select channels of interest

We want to transpose the array so that the channel axis is first:

In [160]:
# TODO(template): You may or may not need to do this depending on the initial ordering of the axes of your image.
# The transpose parameter may need to be changed depending on the number of dimensions.
img_arr = img_arr.transpose((2, 0, 1))
img_arr.shape

(4, 500, 750)

In [161]:
# TODO(template): You may want to select different channels of interest if your image is multiplexed.
# You will not need to do this if there are no extraneous channels in your image.
img_arr = img_arr[0:3, :, :]
img_arr.shape

(3, 500, 750)

### Save as an OME-TIFF

In this case, the image is RGB so we use `rgb_img_to_ome_tiff`. For multiplexed images, see [multiplex_img_to_ome_tiff](https://vitessce.github.io/vitessce-python/api_data.html#vitessce.data_utils.ome.multiplex_img_to_ome_tiff).

In [167]:
# Create an empty folder for the output file.
os.makedirs(os.path.dirname(PATH_TO_OUTPUT_OME_TIFF), exist_ok=True)
os.makedirs(os.path.dirname(PATH_TO_OUTPUT_PYRAMIDAL_OME_TIFF), exist_ok=True)

rgb_img_to_ome_tiff(
    img_arr,
    output_path=PATH_TO_OUTPUT_OME_TIFF,
    img_name=IMG_NAME,
    axes="CYX",
)

### Convert to a pyramidal OME-TIFF using bfconvert

In [168]:
!~/software/bftools/bfconvert -overwrite -tilex 128 -tiley 128 -pyramid-resolutions 2 -pyramid-scale 2 -compression LZW {PATH_TO_OUTPUT_OME_TIFF} {PATH_TO_OUTPUT_PYRAMIDAL_OME_TIFF}
# TODO(template): For larger images, you will want to comment out the above line and un-comment the line below,
# to increase the tile size (128 -> 512) and the number of pyramid resolutions (2 -> 6).
# !~/software/bftools/bfconvert -tilex 512 -tiley 512 -pyramid-resolutions 6 -pyramid-scale 2 -compression LZW {PATH_TO_OUTPUT_OME_TIFF} {PATH_TO_OUTPUT_PYRAMIDAL_OME_TIFF}

./processed_imaging_data/output.ome.tif
OMETiffReader initializing ./processed_imaging_data/output.ome.tif
Reading IFDs
Populating metadata
[OME-TIFF] -> ./processed_imaging_data/output.pyramid.ome.tif [OME-TIFF]
Tile size = 128 x 128
	Converted 3/3 planes (100%)
Tile size = 128 x 128
	Converted 3/3 planes (100%)
[done]
1.208s elapsed (1.6666666+92.833336ms per plane, 611ms overhead)


## 2. Configure the visualization

In [170]:
vc = VitessceConfig(schema_version="1.0.15", name=CONFIG_NAME, description=CONFIG_DESCRIPTION)

In [171]:
dataset = vc.add_dataset(name=DATASET_NAME).add_object(
    OmeTiffWrapper(img_path=PATH_TO_OUTPUT_PYRAMIDAL_OME_TIFF, name=IMG_NAME)
)

In [172]:
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
lc = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=True)
description = vc.add_view(vt.DESCRIPTION, dataset=dataset)

In [173]:
vc.layout(spatial | (lc / description));

### Render the widget

In [174]:
vc.widget()

VitessceWidget(config={'version': '1.0.15', 'name': 'My config', 'description': 'This dataset reveals...', 'da…

## 3. Export the configuration and data

In [156]:
# TODO(template): The export function does not clear the contents of the `out_dir`.
# You may want to ensure that this folder does not yet exist:
os.makedirs(PATH_TO_EXPORT_DIRECTORY, exist_ok=False)

config_dict = vc.export(to="files", base_url=BASE_URL_PLACEHOLDER, out_dir=PATH_TO_EXPORT_DIRECTORY)

# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(join(PATH_TO_EXPORT_DIRECTORY, "vitessce.json"), "w") as f:
    json.dump(config_dict, f)